In [12]:
import torch

In [13]:
# Adjust the path as necessary to point to your specific split file
split_data_path = '/Users/nash/Project/fedn/fedn/examples/mnist-pytorch/data/clients/5/dataset.pt'
split_data = torch.load(split_data_path)


In [15]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
import time
import transformers
from datasets import Dataset


In [16]:
def load_model():
    model_name='google/flan-t5-base'
    original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return original_model, tokenizer


In [17]:
def load_data(data_path):
    dataset = torch.load(data_path)
    data = Dataset.from_dict(dataset)
    return data

In [18]:
data = load_data(split_data_path)

In [19]:
data

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 2492
})

In [7]:

    lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

    peft_model = get_peft_model(load_model()[0], lora_config)

    working_dir = './'
    output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

    peft_training_args = TrainingArguments(
        output_dir=output_dir,
        auto_find_batch_size=True,
        learning_rate=1e-3, # Higher learning rate than full fine-tuning.
        num_train_epochs=1,
        logging_steps=1,
        max_steps=1    
    )
    
train_sample = load_data(split_data_path)

    
    trainer = Trainer(
        model=peft_model,
        args=peft_training_args,
        train_dataset=train_sample,
        data_collator=transformers.DataCollatorForLanguageModeling(load_model()[1], mlm=False)
    )

    trainer.train()


/Users/nash/Project/fedn/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,0.871100


TrainOutput(global_step=1, training_loss=0.87109375, metrics={'train_runtime': 1199.7709, 'train_samples_per_second': 0.007, 'train_steps_per_second': 0.001, 'total_flos': 5565031907328.0, 'train_loss': 0.87109375, 'epoch': 0.0})